# Read Initial Data

In [9]:
import pandas as pd
time = []
open = []
high = []
low = []
close = []
volume = []
size = -1

def read_df(file_name):

    df = pd.read_feather(file_name)
    df = df.dropna()    # Dropping NA values, so that talib works correctly
    global size
    size = df.shape[0]
    
    global time
    time = df['time'].to_numpy()

    global open
    open = df['open'].to_numpy()

    global high
    high = df['high'].to_numpy()

    global low
    low = df['low'].to_numpy()

    global close
    close = df['close'].to_numpy()

    global volume
    volume = df['volume'].to_numpy()

    return

read_df("data-BNBUSDT1h.feather")

# Create Indicators

In [10]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import talib as ta    # https://github.com/mrjbq7/ta-lib

ma_direction_period = 28
ma_direction = ta.EMA(close, timeperiod=ma_direction_period)
atr_direction_period = 14
atr_direction = ta.ATR(high, low, close, timeperiod=atr_direction_period)

# Generate Trades

In [11]:
in_trade = 0    # +1 = long_entry, -1 = shor_entry

entry_price = -1.0
trades_long = []
trades_short = []

for i in range(size):
    # Long Entry
    if (ma_direction[i] - close[i] > 4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = +1.0
        entry_price = close[i]
    # Long Exit
    if (ma_direction[i] - close[i] < 0.0
        and in_trade == 1.0):
        in_trade = 0.0
        trades_long.append(close[i]/entry_price)

    # Short Entry
    if (ma_direction[i] - close[i] < -4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = -1.0
        entry_price = close[i]
    # Short Exit
    if (ma_direction[i] - close[i] > 0.0
        and in_trade == -1.0):
        in_trade = 0.0
        trades_short.append(entry_price/close[i])


# Calculate Results

In [12]:
long_res = 1.0
for k in trades_long:
    long_res *= k

print("Longs result: ", long_res)
average_long_trades_res = np.sum(trades_long)/len(trades_long)
print("Average Long Trade: ", average_long_trades_res)
DD_long = 1.0 - np.min(trades_long)
print("Highest DD for Longs: ", DD_long)
print()

short_res = 1.0
for k in trades_short:
    short_res *= k

print("Shorts result: ", short_res)
average_short_trades_res = np.sum(trades_short)/len(trades_short)
print("Average Short Trade: ", average_short_trades_res)
DD_short = 1.0 - np.min(trades_short)
print("Highest DD for Shorts: ", DD_short)
print()


DD_highest = None
if DD_long > DD_short:
    DD_highest = DD_long
else: DD_highest = DD_short

score = ((average_long_trades_res + average_short_trades_res) / 2) / DD_highest
print("Score: ", score)

Longs result:  1.1932475343208822
Average Long Trade:  1.0058177152451184
Highest DD for Longs:  0.2489020352179997

Shorts result:  0.16694511027625009
Average Short Trade:  0.9839752741179365
Highest DD for Shorts:  0.4806757383373651

Score:  2.0697872085718907


# Optimize Results

In [ ]:
ma_direction_period = 28

atr_direction_period = 14
